**Table of contents**<a id='toc0_'></a>    
- [4CODE - Projekt Python - Spotify](#toc1_)    
  - [Sprawdzenie danych](#toc1_1_)    
  - [Optymalizacja danych](#toc1_2_)    
  - [Analiza danych](#toc1_3_)    
    - [Analiza całej bazy](#toc1_3_1_)    
      - [Artyści](#toc1_3_1_1_)    
      - [Utwory](#toc1_3_1_2_)    
    - [Analiza listy Global](#toc1_3_2_)    
      - [Artyści](#toc1_3_2_1_)    
      - [Utwory](#toc1_3_2_2_)    
    - [Analiza Polska](#toc1_3_3_)    
      - [Artyści](#toc1_3_3_1_)    
      - [Utwory](#toc1_3_3_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[4CODE - Projekt Python - Spotify](#toc0_)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Wczytanie danych
spotify = pd.read_csv("Database to calculate popularity.csv", index_col='Unnamed: 0')

## <a id='toc1_1_'></a>[Sprawdzenie danych](#toc0_)

In [19]:
# Wyświetlenie pierwszych 5 wierszy danych
spotify.head()

,country,position,track,title,artist,year,month,day
0,Global,1,"Dakiti , Bad Bunny, Jhay Cortez",Dakiti,"Bad Bunny, Jhay Cortez",2020,5,11
1,Global,2,"positions , Ariana Grande",positions,Ariana Grande,2020,5,11
2,Global,3,"Mood , 24kGoldn",Mood,24kGoldn,2020,5,11
3,Global,4,"Lemonade , Internet Money",Lemonade,Internet Money,2020,5,11
4,Global,5,"Lonely , Justin Bieber",Lonely,Justin Bieber,2020,5,11


In [20]:
# Wyświetlenie ostatnich 5 wierszy danych
spotify.tail()

,country,position,track,title,artist,year,month,day
9806995,Taiwan,196,"Oh Lord , MiC LOWRY",Oh Lord,MiC LOWRY,2017,1,1
9806996,Taiwan,197,"Love Me Like You Do - From ""Fifty Shades Of G...","Love Me Like You Do - From ""Fifty Shades Of G...",Ellie Goulding,2017,1,1
9806997,Taiwan,198,"Stressed Out , Twenty One Pilots",Stressed Out,Twenty One Pilots,2017,1,1
9806998,Taiwan,199,"The End - From ""La La Land"" Soundtrack , Just...","The End - From ""La La Land"" Soundtrack",Justin Hurwitz,2017,1,1
9806999,Taiwan,200,"Light It Up - Remix , Major Lazer",Light It Up - Remix,Major Lazer,2017,1,1


In [3]:
# Wyświetlenie informacji o data frame
spotify.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9807001 entries, 0 to 9807000
Data columns (total 7 columns):
 #   Column    Dtype  
---  ------    -----  
 0   country   object 
 1   date      object 
 2   position  float64
 3   uri       object 
 4   track     object 
 5   title     object 
 6   artist    object 
dtypes: float64(1), object(6)
memory usage: 4.6 GB


## <a id='toc1_2_'></a>[Optymalizacja danych](#toc0_)

In [3]:
# Usunięcie kolumny 'uri', która nie jest potrzebna do analizy
spotify.drop(columns=['uri'], inplace=True)

# Usunięcie ostatniego wiersza NaN, która nie jest potrzebna do analizy
spotify.drop(spotify.index[-1], inplace=True)

# Zamiana kolumny 'date' na typ datetime
spotify['date'] = pd.to_datetime(spotify['date'])

# Dodanie kolumny 'year' na podstawie kolumny 'date'
spotify['year'] = spotify['date'].dt.year

# Dodanie kolumny 'month' na podstawie kolumny 'date'
spotify['month'] = spotify['date'].dt.month

# Dodanie kolumny 'day' na podstawie kolumny 'date'
spotify['day'] = spotify['date'].dt.day

# Usunięcie kolumny 'date', która nie jest potrzebna do analizy
spotify.drop(columns=['date'], inplace=True)

# Usuwanie wartości NaN z kolumn
spotify.dropna(subset=['position'], inplace=True)
spotify.dropna(subset=['day'], inplace=True)
spotify.dropna(subset=['month'], inplace=True)
spotify.dropna(subset=['year'], inplace=True)

# Zamiana kolumn liczbowych na int
spotify['position'] = spotify['position'].astype(int)
spotify['day'] = spotify['day'].astype(int)
spotify['month'] = spotify['month'].astype(int)
spotify['year'] = spotify['year'].astype(int)

# Zamiana kolumn object na category
spotify.country = spotify.country.astype('category')
spotify.track = spotify.track.astype('category')
spotify.title = spotify.title.astype('category')
spotify.artist = spotify.artist.astype('category')

# Usunięcie duplikatów
spotify.drop_duplicates(inplace=True)

/var/folders/4d/b5zn6jj10d3bz3jfx1x7rggw0000gn/T/ipykernel_1728/3082417695.py:8: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  spotify['date'] = pd.to_datetime(spotify['date'])


In [16]:
spotify.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9807000 entries, 0 to 9806999
Data columns (total 8 columns):
 #   Column    Dtype   
---  ------    -----   
 0   country   category
 1   position  int64   
 2   track     category
 3   title     category
 4   artist    category
 5   year      int64   
 6   month     int64   
 7   day       int64   
dtypes: category(4), int64(4)
memory usage: 491.1 MB


In [4]:
# Wyświetlenie pierwszych 5 wierszy danych
spotify.head()

,country,position,track,title,artist,year,month,day
0,Global,1,"Dakiti , Bad Bunny, Jhay Cortez",Dakiti,"Bad Bunny, Jhay Cortez",2020,5,11
1,Global,2,"positions , Ariana Grande",positions,Ariana Grande,2020,5,11
2,Global,3,"Mood , 24kGoldn",Mood,24kGoldn,2020,5,11
3,Global,4,"Lemonade , Internet Money",Lemonade,Internet Money,2020,5,11
4,Global,5,"Lonely , Justin Bieber",Lonely,Justin Bieber,2020,5,11


In [5]:
# Wyświetlenie ostatnich 5 wierszy danych
spotify.tail()

,country,position,track,title,artist,year,month,day
9806995,Taiwan,196,"Oh Lord , MiC LOWRY",Oh Lord,MiC LOWRY,2017,1,1
9806996,Taiwan,197,"Love Me Like You Do - From ""Fifty Shades Of G...","Love Me Like You Do - From ""Fifty Shades Of G...",Ellie Goulding,2017,1,1
9806997,Taiwan,198,"Stressed Out , Twenty One Pilots",Stressed Out,Twenty One Pilots,2017,1,1
9806998,Taiwan,199,"The End - From ""La La Land"" Soundtrack , Just...","The End - From ""La La Land"" Soundtrack",Justin Hurwitz,2017,1,1
9806999,Taiwan,200,"Light It Up - Remix , Major Lazer",Light It Up - Remix,Major Lazer,2017,1,1


## <a id='toc1_3_'></a>[Analiza danych](#toc0_)

### <a id='toc1_3_1_'></a>[Analiza całej bazy](#toc0_)

In [29]:
spotify.position.describe()

count    9.807000e+06
mean     1.005000e+02
std      5.773431e+01
min      1.000000e+00
25%      5.075000e+01
50%      1.005000e+02
75%      1.502500e+02
max      2.000000e+02
Name: position, dtype: float64

In [30]:
# Ilość krajów w bazie
total_countries = spotify['country'].nunique()
print(f"Liczba krajów w bazie: {total_countries}")

Liczba krajów w bazie: 35


In [31]:
# Ilość artystów w bazie
total_artists = spotify['artist'].nunique()
print(f"Liczba unikalnych artystów w bazie: {total_artists}")

Liczba unikalnych artystów w bazie: 21469


In [32]:
# Ilość utworów w bazie
total_tracks = spotify['track'].nunique()
print(f"Liczba utworów w bazie: {total_tracks}")

Liczba utworów w bazie: 56660


#### <a id='toc1_3_1_1_'></a>[Artyści](#toc0_)

In [33]:
# Najpopularniejsi artyści na świecie w 2017
top_artists_2017 = spotify.loc[(spotify['year'] == 2017)].groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na świecie w 2017 roku:")
print(top_artists_2017)

Top 10 najpopularniejszych artystów na świecie w 2017 roku:
artist
 Ed Sheeran           8190919
 Major Lazer          2581441
 Shawn Mendes         2501379
 The Chainsmokers     2165221
 Drake                2118930
 Bruno Mars           2106018
 Justin Bieber        2080715
 Kendrick Lamar       1949635
 Coldplay             1947743
 Sia                  1867478
Name: position, dtype: int64


In [34]:
# Najpopularniejsi artyści na świecie w 2018
top_artists_2018 = spotify.loc[(spotify['year'] == 2018)].groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na świecie w 2018 roku:")
print(top_artists_2018)

Top 10 najpopularniejszych artystów na świecie w 2018 roku:
artist
 Ed Sheeran          5867633
 XXXTENTACION        4794845
 Post Malone         3811340
 Imagine Dragons     3403767
 Drake               2678260
 Eminem              1960081
 Shawn Mendes        1955906
 Dua Lipa            1925108
 Camila Cabello      1896539
 Khalid              1767754
Name: position, dtype: int64


In [35]:
# Najpopularniejsi artyści na świecie w 2019
top_artists_2019 = spotify.loc[(spotify['year'] == 2019)].groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na świecie w 2019 roku:")
print(top_artists_2019)

Top 10 najpopularniejszych artystów na świecie w 2019 roku:
artist
 Billie Eilish       5253631
 Ed Sheeran          4903187
 Post Malone         3705951
 Queen               2759009
 Ariana Grande       2476833
 Khalid              2135392
 Imagine Dragons     2085464
 XXXTENTACION        1982778
 Juice WRLD          1545276
 Bad Bunny           1483970
Name: position, dtype: int64


In [36]:
# Najpopularniejsi artyści na świecie w 2020
top_artists_2020 = spotify.loc[(spotify['year'] == 2020)].groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na świecie w 2020 roku:")
print(top_artists_2020)

Top 10 najpopularniejszych artystów na świecie w 2020 roku:
artist
 Billie Eilish     3571967
 Juice WRLD        2645404
 Ed Sheeran        2639187
 Travis Scott      2144542
 Lewis Capaldi     1946826
 Harry Styles      1946725
 Dua Lipa          1750447
 Pop Smoke         1737366
 Post Malone       1705111
 Bad Bunny         1406474
Name: position, dtype: int64


In [37]:
# Najpopularniejsi artyści na świecie w 2017 - 2020
top_artists = spotify.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na świecie w latach 2017 - 2020:")
print(top_artists)

Top 10 najpopularniejszych artystów na świecie w latach 2017 - 2020:
artist
 Ed Sheeran          21600926
 Post Malone         10218977
 Billie Eilish        9994756
 XXXTENTACION         8988373
 Imagine Dragons      7918155
 Drake                7302590
 Shawn Mendes         6134582
 Travis Scott         5784227
 Ariana Grande        5441118
 Dua Lipa             5294146
Name: position, dtype: int64


#### <a id='toc1_3_1_2_'></a>[Utwory](#toc0_)

In [38]:
# Najpopularniejsze utwory na świecie w danym 2017 roku
top_songs_2017 = spotify.loc[(spotify['year'] == 2017)].groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na świecie w 2017 roku:")
print(top_songs_2017)

Top 10 najpopularniejszych utworów na świecie w 2017 roku:
track
 Thinking out Loud , Ed Sheeran                        1157016
 Photograph , Ed Sheeran                               1140227
 Treat You Better , Shawn Mendes                        999389
 Love Yourself , Justin Bieber                          977049
 Don't Let Me Down  , The Chainsmokers                  972399
 We Don't Talk Anymore  , Charlie Puth                  925471
 I Took A Pill In Ibiza - Seeb Remix , Mike Posner      909839
 One Dance , Drake, WizKid, Kyla                        902174
 Cheap Thrills , Sia                                    880081
 Let Me Love You , DJ Snake, Justin Bieber              856255
Name: position, dtype: int64


In [39]:
# Najpopularniejsze utwory na świecie 2018 roku
top_songs_2018 = spotify.loc[(spotify['year'] == 2018)].groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na świecie w 2018 roku:")
print(top_songs_2018)

Top 10 najpopularniejszych utworów na świecie w 2018 roku:
track
 Believer , Imagine Dragons                                1221334
 Thunder , Imagine Dragons                                 1096964
 Perfect , Ed Sheeran                                      1010369
 Shape of You , Ed Sheeran                                  962729
 Havana  , Camila Cabello                                   927416
 New Rules , Dua Lipa                                       924844
 Thinking out Loud , Ed Sheeran                             854877
 Too Good At Good,es , Sam Smith                            818546
 IDGAF , Dua Lipa                                           802453
 Something Just Like This , The Chainsmokers, Coldplay      752035
Name: position, dtype: int64


In [40]:
# Najpopularniejsze utwory na świecie 2019 roku
top_songs_2019 = spotify.loc[(spotify['year'] == 2019)].groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na świecie w 2019 roku:")
print(top_songs_2019)

Top 10 najpopularniejszych utworów na świecie w 2019 roku:
track
 Shape of You , Ed Sheeran                 1092857
 Bohemian Rhapsody - 2011 Mix , Queen      1018476
 Perfect , Ed Sheeran                       933914
 Believer , Imagine Dragons                 877184
 Girls Like You  , Maroon 5                 862990
 Happier , Marshmello, Bastille             818983
 rockstar  , Post Malone                    814705
 when the party's over , Billie Eilish      799822
 Shallow , Lady Gaga, Bradley Cooper        773353
 Don't Stop Me Now - 2011 Mix , Queen       748801
Name: position, dtype: int64


In [41]:
# Najpopularniejsze utwory na świecie 2020 roku
top_songs_2020 = spotify.loc[(spotify['year'] == 2020)].groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na świecie w 2020 roku:")
print(top_songs_2020)

Top 10 najpopularniejszych utworów na świecie w 2020 roku:
track
 Señorita , Shawn Mendes, Camila Cabello     896511
 bad guy , Billie Eilish                     818826
 Shallow , Lady Gaga, Bradley Cooper         777638
 lovely  , Billie Eilish                     773892
 Adore You , Harry Styles                    763248
 I Don't Care  , Ed Sheeran                  715092
 HIGHEST IN THE ROOM , Travis Scott          674380
 Memories , Maroon 5                         668644
 Someone You Loved , Lewis Capaldi           662291
 Ride It , Regard                            646988
Name: position, dtype: int64


In [42]:
# Najpopularniejsze utwory na świecie w latach 2017 - 2020
top_tracks = spotify.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na świecie w latach 2017 - 2020:")
print(top_tracks)

Top 10 najpopularniejszych artystów na świecie w latach 2017 - 2020:
track
 Believer , Imagine Dragons              3371793
 Perfect , Ed Sheeran                    2957517
 Shape of You , Ed Sheeran               2721992
 Say You Won't Let Go , James Arthur     2596215
 Photograph , Ed Sheeran                 2423393
 Thinking out Loud , Ed Sheeran          2405926
 Closer , The Chainsmokers, Halsey       2092139
 Thunder , Imagine Dragons               1962058
 rockstar  , Post Malone                 1869615
 Jocelyn Flores , XXXTENTACION           1856194
Name: position, dtype: int64


### <a id='toc1_3_2_'></a>[Analiza listy Global](#toc0_)

#### <a id='toc1_3_2_1_'></a>[Artyści](#toc0_)

In [43]:
# Najpopularniejsi artyści na liście global w 2017 roku
spotify_2017_glob_art = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2017)]
top_artists_2017_glob = spotify_2017_glob_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na liście globalnej w 2017 roku:")
print(top_artists_2017_glob)

Top 10 najpopularniejszych artystów na liście globalnej w 2017 roku:
artist
 Ed Sheeran            284518
 Drake                 118673
 Kendrick Lamar        117328
 Migos                 107634
 Major Lazer           106553
 Shawn Mendes           95531
 Justin Bieber          93523
 Twenty One Pilots      91287
 XXXTENTACION           87814
 The Chainsmokers       85893
Name: position, dtype: int64


In [44]:
# Najpopularniejsi artyści na liście global w 2018 roku
spotify_2018_glob_art = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2018)]
top_artists_2018_glob = spotify_2018_glob_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na liście globalnej w 2018 roku:")
print(top_artists_2018_glob)

Top 10 najpopularniejszych artystów na liście globalnej w 2018 roku:
artist
 XXXTENTACION        237632
 Ed Sheeran          236285
 Post Malone         180501
 Travis Scott        138462
 Imagine Dragons     116942
 Drake               115022
 Khalid              109216
 Kendrick Lamar       90676
 Eminem               82949
 Shawn Mendes         77638
Name: position, dtype: int64


In [45]:
# Najpopularniejsi artyści na liście global w 2019 roku
spotify_2019_glob_art = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2019)]
top_artists_2019_glob = spotify_2019_glob_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na liście globalnej w 2019 roku:")
print(top_artists_2019_glob)

Top 10 najpopularniejszych artystów na liście globalnej w 2019 roku:
artist
 Billie Eilish       281823
 Ed Sheeran          212668
 XXXTENTACION        186950
 Post Malone         183206
 Queen               145930
 Imagine Dragons     125406
 Drake               108778
 Khalid              108504
 Ariana Grande       100065
 Juice WRLD           88677
Name: position, dtype: int64


In [46]:
# Najpopularniejsi artyści na liście global w 2020 roku
spotify_2020_glob_art = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2020)]
top_artists_2020_glob = spotify_2020_glob_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na liście Global w 2020 roku:")
print(top_artists_2020_glob)

Top 10 najpopularniejszych artystów na liście globalnej w 2020 roku:
artist
 Juice WRLD        174593
 Ed Sheeran        171732
 Billie Eilish     165541
 XXXTENTACION      133360
 Post Malone       113457
 Travis Scott       94809
 J Balvin           91205
 Harry Styles       86737
 Pop Smoke          83015
 Bad Bunny          80950
Name: position, dtype: int64


In [67]:
# Najpopularniejsi artyści na liście globa w 2017 - 2020
spotify_glob_art = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2020)]
top_artists_glob = spotify_glob_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów na liście Global w latach 2017 - 2020:")
print(top_artists_glob)

Top 10 najpopularniejszych artystów na liście global w latach 2017 - 2020:
artist
 Juice WRLD        174593
 Ed Sheeran        171732
 Billie Eilish     165541
 XXXTENTACION      133360
 Post Malone       113457
 Travis Scott       94809
 J Balvin           91205
 Harry Styles       86737
 Pop Smoke          83015
 Bad Bunny          80950
Name: position, dtype: int64


#### <a id='toc1_3_2_2_'></a>[Utwory](#toc0_)

In [48]:
# Najpopularniejsze utwory na liście global w 2017 roku:
spotify_2017_glob = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2017)]
top_songs_2017_glob = spotify_2017_glob.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na liście Global w 2017 roku:")
print(top_songs_2017_glob)

Top 10 najpopularniejszych utworów na liście Global w 2017 roku:
track
 White Iverson , Post Malone                           51020
 Lose Yourself - From "8 Mile" Soundtrack , Eminem     48871
 Love Yourself , Justin Bieber                         47765
 Thinking out Loud , Ed Sheeran                        45742
 We Don't Talk Anymore  , Charlie Puth                 42390
 Photograph , Ed Sheeran                               39761
 Treat You Better , Shawn Mendes                       38839
 Happier , Ed Sheeran                                  38147
 Look At Me! , XXXTENTACION                            38023
 Me llamas  - Remix , Piso 21                          37379
Name: position, dtype: int64


In [49]:
# Najpopularniejsze utwory na liście global w 2018 roku:
spotify_2018_glob = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2018)]
top_songs_2018_glob = spotify_2018_glob.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na liście Global w 2018 roku:")
print(top_songs_2018_glob)

Top 10 najpopularniejszych utworów na liście Global w 2018 roku:
track
 Thinking out Loud , Ed Sheeran                            52811
 Closer , The Chainsmokers, Halsey                         51187
 goosebumps , Travis Scott                                 49284
 Wonderwall - Remastered , Oasis                           47647
 Say You Won't Let Go , James Arthur                       45731
 Something Just Like This , The Chainsmokers, Coldplay     45567
 Photograph , Ed Sheeran                                   45199
 I Like Me Better , Lauv                                   45007
 Africa , TOTO                                             42029
 Whatever It Takes , Imagine Dragons                       41320
Name: position, dtype: int64


In [50]:
# Najpopularniejsze utwory na liście global w 2019 roku:
spotify_2019_glob = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2019)]
top_songs_2019_glob = spotify_2019_glob.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na liście Global w 2019 roku:")
print(top_songs_2019_glob)

Top 10 najpopularniejszych utworów na liście Global w 2019 roku:
track
 Closer , The Chainsmokers, Halsey                        51074
 Say You Won't Let Go , James Arthur                      49511
 Photograph , Ed Sheeran                                  49495
 Thunder , Imagine Dragons                                49222
 Too Good At Good,es , Sam Smith                          48468
 Wonderwall - Remastered , Oasis                          47941
 Jocelyn Flores , XXXTENTACION                            46947
 I Like Me Better , Lauv                                  45718
 Another One Bites The Dust - Remastered 2011 , Queen     44786
 Moonlight , XXXTENTACION                                 43758
Name: position, dtype: int64


In [51]:
# Najpopularniejsze utwory na liście global w 2020 roku:
spotify_2020_glob = spotify.loc[(spotify['country'] == 'Global') & (spotify['year'] == 2020)]
top_songs_2020_glob = spotify_2020_glob.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów na liście Global w 2020 roku:")
print(top_songs_2020_glob)

Top 10 najpopularniejszych utworów na liście Global w 2020 roku:
track
 All of Me , John Legend                      49273
 Perfect , Ed Sheeran                         44139
 Moonlight , XXXTENTACION                     43047
 Without Me , Halsey                          42714
 The Less I Know The Better , Tame Impala     42043
 Get You The Moon  , Kina                     41654
 Be Alright , Dean Lewis                      41464
 SAD! , XXXTENTACION                          41394
 Say You Won't Let Go , James Arthur          41188
 Jocelyn Flores , XXXTENTACION                40402
Name: position, dtype: int64


In [69]:
# Najpopularniejsze utwory na liście global w latach 2017 - 2020
top_songs_glob = spotify.loc[(spotify['country'] == 'Global')].groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych na loiście global:")
print(top_songs_glob)

Top 10 najpopularniejszych na loiście global:
track
 Say You Won't Let Go , James Arthur                       157424
 Photograph , Ed Sheeran                                   149311
 Closer , The Chainsmokers, Halsey                         141807
 goosebumps , Travis Scott                                 136607
 Something Just Like This , The Chainsmokers, Coldplay     134956
 Thinking out Loud , Ed Sheeran                            131101
 I Like Me Better , Lauv                                   129551
 Perfect , Ed Sheeran                                      128675
 Wonderwall - Remastered , Oasis                           121671
 Jocelyn Flores , XXXTENTACION                             119226
Name: position, dtype: int64


### <a id='toc1_3_3_'></a>[Analiza Polska](#toc0_)

#### <a id='toc1_3_3_1_'></a>[Artyści](#toc0_)

In [70]:
# Najpopularniejsi artyści w Polsce w 2017 roku
spotify_2017_pl_art = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2017)]
top_artists_2017_pl = spotify_2017_pl_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów w Polsce w 2017 roku:")
print(top_artists_2017_pl)

Top 10 najpopularniejszych artystów w Polsce w 2017 roku:
artist
 Taco Hemingway        346598
 Ed Sheeran            272722
 Quebonafide           143544
 Imagine Dragons       141496
 Major Lazer           115489
 Twenty One Pilots      94857
 Shawn Mendes           92850
 Sia                    83337
 Dawid Podsiadło        72543
 The Weeknd             72050
Name: position, dtype: int64


In [54]:
# Najpopularniejsi artyści w Polsce w 2018 roku
spotify_2018_pl_art = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2018)]
top_artists_2018_pl = spotify_2018_pl_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów w Polsce w 2018 roku:")
print(top_artists_2018_pl)

Top 10 najpopularniejszych artystów w Polsce w 2018 roku:
artist
 Taco Hemingway                              270252
 TACONAFIDE, Quebonafide, Taco Hemingway     169206
 XXXTENTACION                                135663
 Imagine Dragons                             131329
 Dawid Podsiadło                             114459
 Young Igi                                    99131
 Bedoes, Kubi Producent                       95244
 Ed Sheeran                                   89201
 Post Malone                                  84874
 Shawn Mendes                                 79854
Name: position, dtype: int64


In [55]:
# Najpopularniejsi artyści w Polsce w 2019 roku
spotify_2019_pl_art = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2019)]
top_artists_2019_pl = spotify_2019_pl_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów w Polsce w 2019 roku:")
print(top_artists_2019_pl)

Top 10 najpopularniejszych artystów w Polsce w 2019 roku:
artist
 Dawid Podsiadło     213079
 Billie Eilish       180540
 chillwagon          134847
 PRO8L3M             126796
 White 2115          124180
 Szpaku              113581
 Otsochodzi          109559
 Taco Hemingway       98938
 Post Malone          95435
 Guzior               91264
Name: position, dtype: int64


In [56]:
# Najpopularniejsi artyści w Polsce w 2020 roku
spotify_2020_pl_art = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2020)]
top_artists_2020_pl = spotify_2020_pl_art.groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów w Polsce w 2020 roku:")
print(top_artists_2020_pl)

Top 10 najpopularniejszych artystów w Polsce w 2020 roku:
artist
 Bedoes, Lanek       159857
 Mata                140657
 White 2115          127273
 Dawid Podsiadło     108975
 chillwagon          105312
 Billie Eilish        95997
 Taco Hemingway       90794
 PRO8L3M              85905
 Ava Max              81964
 Żabson               81954
Name: position, dtype: int64


In [57]:
# Najpopularniejsi artyści w Polsce w latach 2017 - 2020
top_artists_pl = spotify.loc[(spotify['country'] == 'Poland')].groupby('artist')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych artystów w Polsce:")
print(top_artists_pl)

Top 10 najpopularniejszych artystów w Polsce:
artist
 Taco Hemingway                              806582
 Dawid Podsiadło                             509056
 Ed Sheeran                                  479494
 Imagine Dragons                             352317
 Billie Eilish                               326107
 White 2115                                  303236
 TACONAFIDE, Quebonafide, Taco Hemingway     299911
 PRO8L3M                                     277790
 chillwagon                                  240159
 XXXTENTACION                                237818
Name: position, dtype: int64


#### <a id='toc1_3_3_2_'></a>[Utwory](#toc0_)

In [58]:
# Najpopularniejsze utwory w Polsce w 2017 roku:
spotify_2017_pl = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2017)]
top_songs_2017_pl = spotify_2017_pl.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów w Polsce w 2017 roku:")
print(top_songs_2017_pl)

Top 10 najpopularniejszych utworów w Polsce w 2017 roku:
track
 Gdy,ś Kiedyś , Paluch                     51205
 Szaman , Paluch                           49710
 Do I Wanna Know? , Arctic Monkeys         48896
 6 Zer , Taco Hemingway                    48592
 Seven Nation Army , The White Stripes     48249
 Riptide , Vance Joy                       46945
 Następna stacja , Taco Hemingway          45349
 Demons , Imagine Dragons                  45320
 W Dobrą Stronę , Dawid Podsiadło          42640
 Marmur , Taco Hemingway                   42396
Name: position, dtype: int64


In [59]:
# Najpopularniejsze utwory w Polsce w 2018 roku:
spotify_2018_pl = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2018)]
top_songs_2018_pl = spotify_2018_pl.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów w Polsce w 2018 roku:")
print(top_songs_2018_pl)

Top 10 najpopularniejszych utworów w Polsce w 2018 roku:
track
 Should I Stay or Should I Go - Remastered , The Clash     39713
 W Dobrą Stronę , Dawid Podsiadło                          39405
 Pastempomat , Dawid Podsiadło                             37819
 XO Tour Llif3 , Lil Uzi Vert                              35738
 Mówiłaś , Young Igi                                       35661
 Thunder , Imagine Dragons                                 35503
 Papierosy , Reto                                          35033
 Biały i młody , Bedoes, Kubi Producent                    33265
 Seven Nation Army , The White Stripes                     32571
 MINIMINI , 2K, Szpaku                                     32254
Name: position, dtype: int64


In [60]:
# Najpopularniejsze utwory w Polsce w 2019 roku:
spotify_2019_pl = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2019)]
top_songs_2019_pl = spotify_2019_pl.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów w Polsce w 2019 roku:")
print(top_songs_2019_pl)

Top 10 najpopularniejszych utworów w Polsce w 2019 roku:
track
 Noc , White 2115                            47142
 Płuca zlepione topami , Guzior              46794
 Układanka , Jan-Rapowanie, Nocny, Holak     45003
 WIFI , Guzior                               44470
 Dżins , Dawid Podsiadło                     41921
 Hinata , Szpaku                             40798
 Half Dead , Quebonafide, Reto               39985
 UFO , Szpaku                                39419
 MINIMINI , 2K, Szpaku                       38555
 Nie / nie , Otsochodzi                      37580
Name: position, dtype: int64


In [61]:
# Najpopularniejsze utwory w Polsce w 2020 roku:
spotify_2020_pl = spotify.loc[(spotify['country'] == 'Poland') & (spotify['year'] == 2020)]
top_songs_2020_pl = spotify_2020_pl.groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów w Polsce w 2020 roku:")
print(top_songs_2020_pl)

Top 10 najpopularniejszych utworów w Polsce w 2020 roku:
track
 Noc , White 2115                    47562
 Grill u Gawrona , Białas, Lanek     47157
 Czempion , Kizo, Malik Montana      45387
 UA , Reto                           40079
 Wampir , Sarius                     39793
 Casablanca  , BLACHA                39595
 Hinata , Szpaku                     39329
 koszmar minionego lata , Kartky     38945
 Płuca zlepione topami , Guzior      38563
 Before You Go , Lewis Capaldi       38479
Name: position, dtype: int64


In [62]:
#  Najpopularniejsze utwory w Polsce w latach 2017 - 2020
top_songs_2020_pl = spotify.loc[(spotify['country'] == 'Poland')].groupby('track')['position'].sum().sort_values(ascending=False).head(10)
print("Top 10 najpopularniejszych utworów w Polsce:")
print(top_songs_2020_pl)

Top 10 najpopularniejszych utworów w Polsce:
track
 Noc , White 2115                                         108366
 Płuca zlepione topami , Guzior                           100811
 W Dobrą Stronę , Dawid Podsiadło                          97251
 05:05 , Bedoes, Kubi Producent                            91287
 Half Dead , Quebonafide, Reto                             87675
 UA , Reto                                                 83559
 lovely  , Billie Eilish                                   82044
 Hinata , Szpaku                                           81795
 Tamagotchi , TACONAFIDE, Quebonafide, Taco Hemingway      81644
 Seven Nation Army , The White Stripes                     80820
Name: position, dtype: int64
